In [1]:
from PIL import Image, ImageDraw
import numpy as np
import copy

In [2]:
def display_np(*imgs):
    """
    Display a numpy array as an image
    """
    for im in imgs:
        result = Image.fromarray(im.astype('uint8'))
        result.show()

In [3]:
def check_range(*x): 
    for x_ in x:
        print(np.min(x_), np.max(x_))

In [4]:
def read_image(x, mode = None):
    x = Image.open(x).convert(mode)
    x = np.array(x)
    return x

In [5]:
def normalize(input_im): 
    base = input_im.min() 
    roof = input_im.max() 
    diff = roof - base 
    scale = diff /255

    input_im = input_im - base 
    output = input_im/scale
    return output

In [6]:
im = read_image('trial_2.png', 'L')

In [7]:
display_np(im)

In [8]:
def conv1d(image, kernel, order = 'C', div = 1, norm = True):
    """
    Performs 1d convolution on an image
    """
    
    im = copy.deepcopy(image.flatten())
    k = copy.deepcopy(kernel.flatten(order = order))

    size = len(k)
    to_remove = len(k) - 1
    
    output = np.full((im.shape), 255)
    for i in range(len(im) - to_remove):
        output[i] = np.sum(k * im[i:i+size]) / div
    output = output.reshape(image.shape)
    if norm: output = normalize(output)
    return output

In [9]:
dx = np.array([-1, 1])

In [10]:
Ix = conv1d(im, dx)
Iy = conv1d(im.T, dx).T

In [11]:
check_range(Ix, Iy)

0.0 255.0
0.0 254.99999999999997


In [12]:
display_np(Ix, Iy)

In [13]:
Ix2 = normalize(Ix * Ix)
Ixy = normalize(Ix * Iy)
Iy2 = normalize(Iy * Iy)

In [14]:
check_range(Ix2, Ixy, Iy2)

0.0 255.0
0.0 255.0
0.0 255.0


In [15]:
display_np(Ix2, Ixy, Iy2)

In [16]:
def conv2d(image, kernel, div = 1, norm = True):
    """
    Applies 2d convolution on a 2d image. Also works with rectangular kernels
    """

    # number of rows and columns of the kernel
    r = kernel.shape[0]
    c = kernel.shape[1]

    # initialize a canvas for the output with 255s. We will fill values in this
    output = np.full(image.shape, 255)
    for i in range(image.shape[0] - r - 1):
        for j in range(image.shape[1] - c - 1):
            output[i][j] = np.sum(kernel * image[i:i+r, j:j+c]) / div
    if norm: output = normalize(output)
    return output

In [17]:
b = np.ones((3,3))
b

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [1]:
# Aw = conv2d(Ix2, b)
# Bw = conv2d(Ixy, b)
# Cw = conv2d(Iy2, b)

In [19]:
check_range(Aw, Bw, Cw)

0.0 255.0
0.0 255.0
0.0 255.0


In [20]:
# display_np(Aw, Bw, Cw)

In [21]:
Aw.shape, Bw.shape, Cw.shape, im.shape

((824, 824), (824, 824), (824, 824), (824, 824))

In [27]:
T = 5.3
op_img = Image.open('trial_2.png')
draw = ImageDraw.Draw(op_img)
for i in range(im.shape[0]):
    for j in range(im.shape[1]):
        eigv, eigvec = np.linalg.eig(np.array([[Aw[i,j], Bw[i,j]],
                                               [Bw[i,j], Cw[i,j]]]))
        m = min(eigv)
        if m > T:
            draw.line(((j-5,i) ,(j+5,i)), fill =(255,0,0))
            draw.line(((j,i-5),(j,i+5)), fill=(255,0,0))
op_img.show()